## 主函数

In [1]:
# -*- coding: utf-8 -*-
"""
双流端到端自动驾驶网络 (Dual-Stream End-to-End Driving Model)
- 视觉流: ResNet_CBAM + Transformer (处理 t-2, t-1, t 三帧图像)
- 状态流: LSTM (处理过去 N 帧的 [速度, 加速度, 转角])
- 融合策略: Output = MLP(Concat(Visual, LSTM)) + LSTM
- 预测输出: 当前时刻所需的 [加速度, 转角]
"""

# 还原数据
# target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']


处理完成！归一化参数已保存至 scaler_params.json，序列化数据已保存至 global_vehicle_data_history_cols.csv


## 对速度进行预测

In [1]:
# 加入了超参数
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from dataset_create import prepare_multiple_datasets_and_scaler, ProcessedDrivingDataset, inverse_transform
from model import DualStreamDrivingModel
from datetime import datetime

def train():
    # 配置文件路径
    input_files = ['carla_data_collect/20260130_164858/csv/global_vehicle_data.csv',
                'carla_data_collect/20260131_203749/csv/global_vehicle_data.csv',
                 'carla_data_collect/20260131_203911/csv/global_vehicle_data.csv',
                 'carla_data_collect/20260131_204009/csv/global_vehicle_data.csv',
                 'carla_data_collect/20260131_204340/csv/global_vehicle_data.csv']
    output_csv = 'processed_data.csv'
    scaler_json_path = 'scaler_params.json'
    target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']

    # 数据预处理
    print(">>> 开始检查并预处理数据...")
    scaler_params = prepare_multiple_datasets_and_scaler(
        input_files=input_files,
        output_csv=output_csv,
        scaler_json_path=scaler_json_path,
        seq_length=9
    )
    print(">>> 预处理完成！\n")

    # 加载数据集
    dataset = ProcessedDrivingDataset(csv_file=output_csv)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)

    # 初始化模型、损失函数、优化器
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DualStreamDrivingModel().to(device)
    optimizer = Adam(model.parameters(), lr=1e-4)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    # 设置加权损失的权重超参数
    a = 0.3  # 速度的权重，转角的权重为 1-a

    # 日志文件
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

    # 训练循环
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for batch_idx, (images, state_seq, target) in enumerate(dataloader):
            # 数据移至设备
            img_t_minus_2, img_t_minus_1, img_t = [img.to(device) for img in images]
            state_seq = state_seq.to(device)
            target = target.to(device)

            # 前向传播
            predictions = model(img_t_minus_2, img_t_minus_1, img_t, state_seq)

            # 计算加权损失
            # acceleration_loss = nn.MSELoss()(predictions[:, :3], target[:, :3])  # 加速度损失
            # steer_loss = nn.MSELoss()(predictions[:, 3], target[:, 3])           # 转角损失
            # loss = a * acceleration_loss + (1 - a) * steer_loss
            velocity_loss = nn.MSELoss()(predictions[:, 0], target[:, 0])  # 速度损失
            steer_loss = nn.MSELoss()(predictions[:, 1], target[:, 1])    # 转角损失
            loss = a * velocity_loss + (1 - a) * steer_loss

            # 反向传播与优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # 打印当前 Batch 的损失
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

        # 学习率调度
        scheduler.step()

        # 记录日志
        with open(log_file, "a") as f:
            f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}\n")

        print(f"Epoch [{epoch+1}/{num_epochs}] 完成，平均 Loss: {epoch_loss/len(dataloader):.4f}")

    # 保存模型
    model_path = os.path.join(log_dir, "v_a0.3.pth")
    torch.save(model.state_dict(), model_path)
    print(f"模型已保存至 {model_path}")

if __name__ == "__main__":
    train()

>>> 开始检查并预处理数据...
>>> 预处理完成！

Epoch [1/20], Batch [1/518], Loss: 0.1076
Epoch [1/20], Batch [2/518], Loss: 0.0320
Epoch [1/20], Batch [3/518], Loss: 0.0327
Epoch [1/20], Batch [4/518], Loss: 0.0684
Epoch [1/20], Batch [5/518], Loss: 0.0173
Epoch [1/20], Batch [6/518], Loss: 0.0493
Epoch [1/20], Batch [7/518], Loss: 0.0200
Epoch [1/20], Batch [8/518], Loss: 0.0434
Epoch [1/20], Batch [9/518], Loss: 0.0238
Epoch [1/20], Batch [10/518], Loss: 0.0482
Epoch [1/20], Batch [11/518], Loss: 0.0484
Epoch [1/20], Batch [12/518], Loss: 0.0197
Epoch [1/20], Batch [13/518], Loss: 0.0173
Epoch [1/20], Batch [14/518], Loss: 0.0185
Epoch [1/20], Batch [15/518], Loss: 0.0140
Epoch [1/20], Batch [16/518], Loss: 0.0408
Epoch [1/20], Batch [17/518], Loss: 0.0290
Epoch [1/20], Batch [18/518], Loss: 0.0188
Epoch [1/20], Batch [19/518], Loss: 0.0078
Epoch [1/20], Batch [20/518], Loss: 0.0325
Epoch [1/20], Batch [21/518], Loss: 0.0242
Epoch [1/20], Batch [22/518], Loss: 0.0137
Epoch [1/20], Batch [23/518], Los

## 对加速度进行预测

In [1]:
# 加入了超参数
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from dataset_create import prepare_multiple_datasets_and_scaler, ProcessedDrivingDataset, inverse_transform
from model import DualStreamDrivingModel
from datetime import datetime

def train():
    # 配置文件路径
    input_csv = 'global_vehicle_data.csv'
    output_csv = 'global_vehicle_data_history_cols.csv'
    scaler_json = 'scaler_params.json'
    target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']

    # 数据预处理
    print(">>> 开始检查并预处理数据...")
    scaler_params = prepare_multiple_datasets_and_scaler(
        input_file=input_csv,
        output_csv=output_csv,
        scaler_json_path=scaler_json,
        seq_length=9
    )
    print(">>> 预处理完成！\n")

    # 加载数据集
    dataset = ProcessedDrivingDataset(csv_file=output_csv)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)

    # 初始化模型、损失函数、优化器
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DualStreamDrivingModel().to(device)
    optimizer = Adam(model.parameters(), lr=1e-4)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    # 设置加权损失的权重超参数
    a = 0.3  # 速度的权重，转角的权重为 1-a

    # 日志文件
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

    # 训练循环
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for batch_idx, (images, state_seq, target) in enumerate(dataloader):
            # 数据移至设备
            img_t_minus_2, img_t_minus_1, img_t = [img.to(device) for img in images]
            state_seq = state_seq.to(device)
            target = target.to(device)

            # 前向传播
            predictions = model(img_t_minus_2, img_t_minus_1, img_t, state_seq)

            # 计算加权损失
            acceleration_loss = nn.MSELoss()(predictions[:, :3], target[:, :3])  # 加速度损失
            steer_loss = nn.MSELoss()(predictions[:, 3], target[:, 3])           # 转角损失
            loss = a * acceleration_loss + (1 - a) * steer_loss

            # 反向传播与优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # 打印当前 Batch 的损失
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

        # 学习率调度
        scheduler.step()

        # 记录日志
        with open(log_file, "a") as f:
            f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}\n")

        print(f"Epoch [{epoch+1}/{num_epochs}] 完成，平均 Loss: {epoch_loss/len(dataloader):.4f}")

    # 保存模型
    model_path = os.path.join(log_dir, "v_a0.3.pth")
    torch.save(model.state_dict(), model_path)
    print(f"模型已保存至 {model_path}")

if __name__ == "__main__":
    train()

>>> 开始检查并预处理数据...
>>> 预处理完成！

Epoch [1/20], Batch [1/778], Loss: 0.3801
Epoch [1/20], Batch [2/778], Loss: 0.1541
Epoch [1/20], Batch [3/778], Loss: 0.0580
Epoch [1/20], Batch [4/778], Loss: 0.0206
Epoch [1/20], Batch [5/778], Loss: 0.0134
Epoch [1/20], Batch [6/778], Loss: 0.0143
Epoch [1/20], Batch [7/778], Loss: 0.0287
Epoch [1/20], Batch [8/778], Loss: 0.0579
Epoch [1/20], Batch [9/778], Loss: 0.0684
Epoch [1/20], Batch [10/778], Loss: 0.0307
Epoch [1/20], Batch [11/778], Loss: 0.0260
Epoch [1/20], Batch [12/778], Loss: 0.0092
Epoch [1/20], Batch [13/778], Loss: 0.0128
Epoch [1/20], Batch [14/778], Loss: 0.0018
Epoch [1/20], Batch [15/778], Loss: 0.0038
Epoch [1/20], Batch [16/778], Loss: 0.0083
Epoch [1/20], Batch [17/778], Loss: 0.0102
Epoch [1/20], Batch [18/778], Loss: 0.0102
Epoch [1/20], Batch [19/778], Loss: 0.0126
Epoch [1/20], Batch [20/778], Loss: 0.0125
Epoch [1/20], Batch [21/778], Loss: 0.0130
Epoch [1/20], Batch [22/778], Loss: 0.0114
Epoch [1/20], Batch [23/778], Los